In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/traffic-signs-preprocessed/data8.pickle
/kaggle/input/traffic-signs-preprocessed/data4.pickle
/kaggle/input/traffic-signs-preprocessed/test.pickle
/kaggle/input/traffic-signs-preprocessed/std_rgb.pickle
/kaggle/input/traffic-signs-preprocessed/data5.pickle
/kaggle/input/traffic-signs-preprocessed/data3.pickle
/kaggle/input/traffic-signs-preprocessed/mean_image_gray.pickle
/kaggle/input/traffic-signs-preprocessed/labels.pickle
/kaggle/input/traffic-signs-preprocessed/data1.pickle
/kaggle/input/traffic-signs-preprocessed/mean_image_rgb.pickle
/kaggle/input/traffic-signs-preprocessed/valid.pickle
/kaggle/input/traffic-signs-preprocessed/data0.pickle
/kaggle/input/traffic-signs-preprocessed/std_gray.pickle
/kaggle/input/traffic-signs-preprocessed/data6.pickle
/kaggle/input/traffic-signs-preprocessed/label_names.csv
/kaggle/input/traffic-signs-preprocessed/datasets_preparing.py
/kaggle/input/traffic-signs-preprocessed/data7.pickle
/kaggle/input/traffic-signs-preprocessed/train

In [2]:
data = pd.read_pickle('/kaggle/input/traffic-signs-preprocessed/data8.pickle')

In [3]:
data

{'y_validation': array([ 9, 37, 31, ...,  5, 18,  6], dtype=uint8),
 'x_train': array([[[[-0.54642206,  1.0425833 ,  0.79494214, ..., -0.70286393,
           -0.67279685, -0.602346  ],
          [-0.6146229 ,  0.74160224,  0.5867728 , ...,  0.16095497,
            0.15466528,  0.0214771 ],
          [-0.69638306,  0.5387884 ,  0.44214383, ..., -0.06861585,
           -0.24066828, -0.35183737],
          ...,
          [ 0.12834986,  0.41358492,  0.5119226 , ...,  0.3618001 ,
            0.545381  , -0.48002478],
          [ 0.4186098 ,  0.38220668,  0.47971445, ...,  0.55423665,
            0.73357224, -0.4668069 ],
          [-0.54892397, -0.61771524, -0.6314567 , ...,  0.68108726,
            0.85835606, -0.4669531 ]]],
 
 
        [[[-0.54642206, -0.56189924, -0.6495985 , ..., -0.12373468,
            0.02414251, -0.54330283],
          [-0.03432735,  0.15697211,  0.10856288, ..., -0.5076013 ,
           -0.14035697, -0.6238451 ],
          [-0.17071831, -0.12555376, -0.0688336 , ..

In [4]:
print(len(data['x_train']),len(data['y_train']),len(data['x_validation']),len(data['y_validation']),len(data['x_test']),len(data['y_test']))


86989 86989 4410 4410 12630 12630


In [5]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import tensorflow as tf

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [6]:
# Normalize image vectors
X_train = data['x_train']/255.
X_test = data['x_test']/255.
X_validation = data['x_validation']/.255

# Reshape
Y_train = data['y_train'].T
Y_test = data['y_test'].T
Y_validation = data['y_validation'].T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of Validation examples = " + str(X_validation.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))

print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))

print ("X_validation shape: " + str(X_validation.shape))
print ("Y_validation shape: " + str(Y_validation.shape))

print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))


number of training examples = 86989
number of Validation examples = 4410
number of test examples = 12630
X_train shape: (86989, 1, 32, 32)
Y_train shape: (86989,)
X_validation shape: (4410, 1, 32, 32)
Y_validation shape: (4410,)
X_test shape: (12630, 1, 32, 32)
Y_test shape: (12630,)


In [7]:
x_validation = X_validation.transpose(0, 2, 3, 1)
x_train = X_train.transpose(0, 2, 3, 1)
x_test = X_test.transpose(0, 2, 3, 1)
print(x_train.shape,x_validation.shape,x_test.shape)

(86989, 32, 32, 1) (4410, 32, 32, 1) (12630, 32, 32, 1)


In [8]:
def model(input_shape):

    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(16, (3, 3), strides = (1, 1), name = 'conv0')(X)
    print(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool0')(X)
    
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool1')(X)
    
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool2')(X)
    

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X)

    
    
    return model

In [9]:
Model = model(x_train.shape[1:])

Tensor("conv0/Identity:0", shape=(None, 36, 36, 16), dtype=float32)


In [10]:
Model.compile(optimizer='Adadelta', loss = 'binary_crossentropy', metrics = ["accuracy"])

In [11]:
Model.fit(x = x_train , y = Y_train, epochs = 50, batch_size = 1000)

Epoch 1/50
87/87 [==============================] - 2s 19ms/step - loss: 5.0870 - accuracy: 0.0250
Epoch 2/50
87/87 [==============================] - 2s 18ms/step - loss: 1.6402 - accuracy: 0.0251
Epoch 3/50
87/87 [==============================] - 2s 18ms/step - loss: -2.0463 - accuracy: 0.0253
Epoch 4/50
87/87 [==============================] - 2s 18ms/step - loss: -5.9466 - accuracy: 0.0252
Epoch 5/50
87/87 [==============================] - 2s 18ms/step - loss: -10.0451 - accuracy: 0.0251
Epoch 6/50
87/87 [==============================] - 2s 18ms/step - loss: -14.3404 - accuracy: 0.0243
Epoch 7/50
87/87 [==============================] - 2s 18ms/step - loss: -18.8122 - accuracy: 0.0238
Epoch 8/50
87/87 [==============================] - 2s 18ms/step - loss: -23.4524 - accuracy: 0.0236
Epoch 9/50
87/87 [==============================] - 2s 18ms/step - loss: -28.2509 - accuracy: 0.0235
Epoch 10/50
87/87 [==============================] - 2s 18ms/step - loss: -33.2076 - accuracy: 0.

In [12]:
preds = Model.evaluate(x = x_validation, y = Y_validation )
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

138/138 [==============================] - 0s 2ms/step - loss: -193977.4531 - accuracy: 0.0544

Loss = -193977.453125
Test Accuracy = 0.054421767592430115


In [13]:
preds = Model.evaluate(x = x_test, y = Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

395/395 [==============================] - 1s 3ms/step - loss: -236.2888 - accuracy: 0.0570

Loss = -236.288818359375
Test Accuracy = 0.05700712651014328
